In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import cv2
from PIL import Image
import os
import pathlib
import random as rn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import argparse
import argparse
import json
import tensorboard
import tensorboardX
import os
import argparse
import json
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim 
import nni
from nni.nas.nn.pytorch import ModelSpace, LayerChoice, MutableConv2d, MutableBatchNorm2d, MutableReLU
from pytorch_lightning import Trainer
from nni.nas.evaluator.pytorch import Lightning, ClassificationModule, Trainer
from nni.nas.experiment import NasExperiment
from nni.nas.space import model_context
from nni.nas.hub.pytorch import DARTS
from nni.nas.strategy import DARTS as DartsStrategy
from pytorch_lightning.loggers import TensorBoardLogger
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import transforms
from torchvision.datasets import CIFAR10
from nni.nas.experiment import NasExperiment
from nni.nas.evaluator import FunctionalEvaluator
from nni.nas.evaluator import FunctionalEvaluator
import nni.nas.strategy as strategy
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
import genotypes
from pytorch_lightning.callbacks import ModelCheckpoint
torch.set_float32_matmul_precision('medium')
from tqdm import tqdm
from nni.nas.nn.pytorch import LayerChoice, ModelSpace,ValueChoice
from torch.utils.data import DataLoader, Dataset, SubsetRandomSampler
from pytorch_lightning import LightningModule, Trainer
from torchvision import datasets, transforms
from nni.nas.evaluator.pytorch import Classification

In [ ]:
def equalize(img):
    #print(type(img), img.shape)
    img1=np.uint8(cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX))
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    img = clahe.apply(img1)
    img = cv2.equalizeHist(img)
    img = tf.expand_dims(img, axis=-1)
    img = np.array(img)
    img = img.astype(np.float64)
    #print(type(img), img.shape)
    return img

In [ ]:
data_dir = 'C:/Users/senti/Desktop/GTSDB'
train_path = 'C:/Users/senti/Desktop/GTSDB/Train'
test_path = 'C:/Users/senti/Desktop/GTSDB/Test/'
height = 32
width = 32

In [ ]:
batch_size = 150
seed = 42

In [ ]:
classes = { 0:'Speed limit (20km/h)',
            1:'Speed limit (30km/h)', 
            2:'Speed limit (50km/h)', 
            3:'Speed limit (60km/h)', 
            4:'Speed limit (70km/h)', 
            5:'Speed limit (80km/h)', 
            6:'End of speed limit (80km/h)', 
            7:'Speed limit (100km/h)', 
            8:'Speed limit (120km/h)', 
            9:'No passing', 
            10:'No passing veh over 3.5 tons', 
            11:'Right-of-way at intersection', 
            12:'Priority road', 
            13:'Yield', 
            14:'Stop', 
            15:'No vehicles', 
            16:'Veh > 3.5 tons prohibited', 
            17:'No entry', 
            18:'General caution', 
            19:'Dangerous curve left', 
            20:'Dangerous curve right', 
            21:'Double curve', 
            22:'Bumpy road', 
            23:'Slippery road', 
            24:'Road narrows on the right', 
            25:'Road work', 
            26:'Traffic signals', 
            27:'Pedestrians', 
            28:'Children crossing', 
            29:'Bicycles crossing', 
            30:'Beware of ice/snow',
            31:'Wild animals crossing', 
            32:'End speed + passing limits', 
            33:'Turn right ahead', 
            34:'Turn left ahead', 
            35:'Ahead only', 
            36:'Go straight or right', 
            37:'Go straight or left', 
            38:'Keep right', 
            39:'Keep left', 
            40:'Roundabout mandatory', 
            41:'End of no passing', 
            42:'End no passing veh > 3.5 tons' }

In [ ]:
import torch
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from torchvision.transforms.functional import equalize


train_transform = transforms.Compose([
    transforms.Resize((32, 32)),  
    transforms.RandomCrop(32, padding=5),
    transforms.RandomHorizontalFlip(),  
    transforms.RandomRotation(15), 
    transforms.Lambda(equalize), 
    transforms.ToTensor(),  
    transforms.Normalize(mean=[0.5], std=[0.5])  
])


val_transform = transforms.Compose([
    transforms.Resize((32, 32)),  # Resize to match training
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])


train_dataset = datasets.ImageFolder(root=train_path, transform=train_transform)
val_dataset = datasets.ImageFolder(root=train_path, transform=val_transform)


print(f"Class to index mapping: {train_dataset.class_to_idx}")


train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])


train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4)
valid_loader = DataLoader(val_dataset, batch_size=16, shuffle=True, num_workers=4)

data_iter = iter(train_loader)
images, labels = next(data_iter)

print(f"Images shape: {images.shape}") 
print(f"Labels shape: {labels.shape}")  
print(f"Labels: {labels}") 


In [ ]:
for i, (inputs, labels) in enumerate(train_loader):
    print(f"Batch {i+1}, Labels: {labels}")


In [ ]:
print(f"Training dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")


In [ ]:
# For train dataset
train_labels = [label for _, label in train_dataset]

# For validation dataset
val_labels = [label for _, label in val_dataset]

In [ ]:
list_set = set(train_labels)
unique_list = (list(list_set))

print(unique_list)
unique_list.append(43)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


data_iter = iter(train_loader)


images, labels = next(data_iter)

fig, ax = plt.subplots(3, 4)
fig.set_size_inches(16, 12)

for i in range(3):
    for j in range(4):
        img = images[i * 4 + j]
        label = labels[i * 4 + j].item() 
        
        ax[i, j].imshow(img.permute(1, 2, 0), cmap='gray') 
        ax[i, j].set_title(f"Label: {classes[label]}") 

plt.tight_layout()
plt.show()


In [ ]:
@nni.trace
class AuxLossClassificationModule(ClassificationModule):
    """Several customization for the training of DARTS, based on default Classification."""
    model: DARTS
    def __init__(self,
                 learning_rate: float = 1e-3,
                 weight_decay: float = 0.,
                 auxiliary_loss_weight: float = 0.4,
                 max_epochs: int = 600):
        print(f"lr : {learning_rate}")
        print(f"weight decay: {weight_decay}")
        print(f"aux loss weight: {auxiliary_loss_weight}")
        print(f"max epochs: {max_epochs}")
        super().__init__(learning_rate=learning_rate, weight_decay=weight_decay, num_classes=43)
        self.auxiliary_loss_weight = auxiliary_loss_weight
        self.max_epochs = max_epochs
        self.criterion=  nn.CrossEntropyLoss()

    def configure_optimizers(self):
        """Customized optimizer with momentum, as well as a scheduler."""
        """
        Classification Module params from nni.nas.evaluators
             learning_rate: float
             weight_decay: float
             optimizer: Type[optim.Optimizer]
             export_onnx: bool 
             num_classes: Optional[int] 

        """
        optimizer = torch.optim.Adam(
            self.parameters(),
            lr = 1e-6,
            betas=(0.9, 0.999), 
            eps=1e-07,
            weight_decay= self.auxiliary_loss_weight 
        )
        return {
            'optimizer': optimizer,
            'lr_scheduler': torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, self.max_epochs, eta_min=1e-3)
        }

    def training_step(self, batch, batch_idx):
        """Training step, customized with auxiliary loss."""
        x, y = batch

        # Check for NaNs or infinite values in input
        if torch.isnan(x).any() or torch.isnan(y).any() or torch.isinf(x).any() or torch.isinf(y).any():
            raise ValueError("Input data contains NaNs or Infinities.")

        if self.auxiliary_loss_weight:
            y_hat, y_aux = self(x)
            loss_main = self.criterion(y_hat, y)
            loss_aux = self.criterion(y_aux, y)
            # Check for NaNs in loss values
            if torch.isnan(loss_main).any() or torch.isnan(loss_aux).any():
                raise ValueError("Loss contains NaNs.")
            self.log('train_loss_main', loss_main)
            self.log('train_loss_aux', loss_aux)
            loss = loss_main + self.auxiliary_loss_weight * loss_aux

        else:
            y_hat = self(x)
            loss = self.criterion(y_hat, y)       
        #acc = (y_hat.argmax(dim=1) == y).float().mean()4 
        self.log('train_loss', loss, prog_bar=True)
        #self.log('train_accuracy', acc, prog_bar=True)  # Log training accuracy
        for name, metric in self.metrics.items():
            self.log('train_' + name, metric(y_hat, y), prog_bar=True)
        return loss
        



    def on_train_epoch_start(self):
        """Set drop path probability before every epoch. This has no effect if drop path is not enabled in model."""
        self.model.set_drop_path_prob(self.model.drop_path_prob * self.current_epoch / self.max_epochs)

        # Logging learning rate at the beginning of every epoch
        self.log('lr', self.trainer.optimizers[0].param_groups[0]['lr'])




In [ ]:

class SepConv(nn.Module):
    def __init__(self, C_in, C_out, kernel_size, stride, padding, affine=True):
        super(SepConv, self).__init__()
        self.op = nn.Sequential(
            MutableConv2d(C_in, C_in, kernel_size=kernel_size, stride=stride, padding=padding),
            MutableConv2d(C_in, C_in, kernel_size=1, padding=0),
            MutableConv2d(C_in, C_in, kernel_size=kernel_size, stride=stride, padding=padding),
            MutableConv2d(C_in, C_out, kernel_size=1, padding=0),
            MutableBatchNorm2d(C_out,affine=affine),
            MutableReLU()
        )
    
    def forward(self, x):
        return self.op(x)
        
                
        
class PhotonicSigmoid(nn.Module):
    def forward(self, x):
        tmp = torch.exp((x - 0.145) / 0.073)
        tmp = 1.005 + (0.06 - 1.005) / (1 + tmp)
        return tmp.float()


In [ ]:

class CustomDARTSSpaceK3(ModelSpace):
    def __init__(self, input_channels, channels, num_classes, layers, verbose):
        super(CustomDARTSSpaceK3, self).__init__()
        #self.first_iter = True
        self.layers = nn.ModuleList()
        self.drop_path_prob = 0.0  
        self.preliminary_layer = nn.Conv2d(3, 8, kernel_size=3, padding=0, bias=False)
        self.verbose = verbose

        layer1_in = 8
        layer1_out = nni.choice('layer1_out_channels', [2,4,8,16,32,64])
        layer2_out= nni.choice('layer2_out_channels', [2,4,8,16,32,64])
        layer3_out= nni.choice('layer3_out_channels', [2,4,8,16,32,64])
        layer4_out= nni.choice('layer4_out_channels', [2,4,8,16,32,64])
        layer5_out = 22 

        
        layer1 = LayerChoice([
              SepConv(8, layer1_out, kernel_size=3, stride=1, padding=1),
            nn.Sequential(
                nn.AvgPool2d(kernel_size=3, stride=1, padding=1),
                MutableConv2d(8, layer1_out, kernel_size=1),
                MutableBatchNorm2d(layer1_out),
                MutableReLU()
            ),   
            nn.Sequential(
                MutableConv2d(8, layer1_out, kernel_size=1),
                nn.AvgPool2d(kernel_size=3, stride=1, padding=1),
                MutableBatchNorm2d(layer1_out),
                MutableReLU()
            )
        ], label='layer_1')
        self.layers.append(layer1)
        
        layer2 = LayerChoice([
            SepConv(layer1_out, layer2_out, kernel_size=3, stride=1, padding=1),
            nn.Sequential(
                nn.AvgPool2d(kernel_size=3, stride=1, padding=1),
                MutableConv2d(layer1_out, layer2_out, kernel_size=1),
                MutableBatchNorm2d(layer2_out),
                MutableReLU()
            ),   
            nn.Sequential(
                MutableConv2d(layer1_out, layer2_out, kernel_size=1),
                nn.AvgPool2d(kernel_size=3, stride=1, padding=1),
                MutableBatchNorm2d(layer2_out),
                MutableReLU()
            )
        ], label='layer_2')
        self.layers.append(layer2)
        
        layer3 = LayerChoice([
            SepConv(layer2_out, layer3_out, kernel_size=3, stride=1, padding=1),
            nn.Sequential(
                nn.AvgPool2d(kernel_size=3, stride=1, padding=1),
                MutableConv2d(layer2_out, layer3_out, kernel_size=1),
                MutableBatchNorm2d(layer3_out),
                MutableReLU()
            ),   
            nn.Sequential(
                MutableConv2d(layer2_out, layer3_out, kernel_size=1),
                nn.AvgPool2d(kernel_size=3, stride=1, padding=1),
                MutableBatchNorm2d(layer3_out),
                MutableReLU()
            )
        ], label='layer_3')
        self.layers.append(layer3)
        
        layer4 = LayerChoice([
            SepConv(layer3_out, layer4_out, kernel_size=3, stride=1, padding=1),
            nn.Sequential(
                nn.AvgPool2d(kernel_size=3, stride=1, padding=1),
                MutableConv2d(layer3_out, layer4_out, kernel_size=1),
                MutableBatchNorm2d(layer4_out),
                MutableReLU()
            ),   
            nn.Sequential(
                MutableConv2d(layer3_out, layer4_out, kernel_size=1),
                nn.AvgPool2d(kernel_size=3, stride=1, padding=1),
                MutableBatchNorm2d(layer4_out),
                MutableReLU()
            )
        ], label='layer_4')
        self.layers.append(layer4)
        
        layer5 = LayerChoice([
            SepConv(layer4_out, layer5_out, kernel_size=3, stride=1, padding=1),
            nn.Sequential(
                nn.AvgPool2d(kernel_size=3, stride=1, padding=1),
                MutableConv2d(layer4_out, layer5_out, kernel_size=1),
                MutableBatchNorm2d(layer5_out),
                MutableReLU()
            ),   
            nn.Sequential(
                MutableConv2d(layer4_out, layer5_out, kernel_size=1),
                nn.AvgPool2d(kernel_size=3, stride=1, padding=1),
                MutableBatchNorm2d(layer5_out),
                MutableReLU()
            )
        ], label='layer_5')
        self.layers.append(layer5)
        
        # Ensure the number of inputs to fc1 is close to but not exceeding 200
        self.pool = nn.AdaptiveAvgPool2d((2, 2))
        self.fc1 = nn.Linear(198, 128) 
        self.fc2 = nn.Linear(128, 64) 
        self.fc3 = nn.Linear(64, 32)  
        self.relu = nn.ReLU()
        self.classifier = nn.Linear(32, num_classes)

    def forward(self, x):
        # Kernel: 3
        # Initial shape: 32x32
        #print(f'Input shape: {x.shape}')
        x = self.preliminary_layer(x)
        # After preliminary layer: 31x31
        if self.verbose == 1 :
            print(f'After preliminary layer: {x.shape}')
        
        for i, layer in enumerate(self.layers):
            x = layer(x)
            if self.verbose == 1 :
                print(f'After layer {i+1}: {x.shape}')
            
            # Add a AvgPool2d layer after the third layer to reduce spatial dimensions
            if i == 2 or i == 4:
                x = nn.AvgPool2d(kernel_size=2, stride=2)(x)
                if self.verbose == 1 :
                    print(f'After avg pooling: {x.shape}')
        

        x =  nn.AvgPool2d(kernel_size=2, stride =2)(x)
        if self.verbose == 1 :
            print(f'After adaptive pooling: {x.shape}')
        
        x = torch.flatten(x, 1)
        if self.verbose == 1 :
            print(f'After flattening: {x.shape}')
        
        x = self.fc1(x)
        x= self.relu(x)
        if self.verbose == 1 :
            print(f'After fc1: {x.shape}')
        x = self.fc2(x)
        x= self.relu(x)
        if self.verbose == 1 :
            print(f'After fc2: {x.shape}')
        x = self.fc3(x)
        x= self.relu(x)
        if self.verbose == 1 :
            print(f'After fc3: {x.shape}')
        
        x = self.classifier(x)
        if self.verbose == 1 :
            print(f'After classifier: {x.shape}')
        #self.first_iter = False
        return x

    def set_drop_path_prob(self, drop_path_prob):
        self.drop_path_prob = drop_path_prob
        for layer in self.layers:
            if hasattr(layer, 'set_drop_path_prob'):
                layer.set_drop_path_prob(drop_path_prob)


In [ ]:
def search(log_dir: str, batch_size: int = 16):
    """
    Darts search 

    Args:
        log_dir (str): The directory where logs will be saved.
        batch_size (int, optional): The size of the batches. Default is 64.  
    Returns:
        None
    """

    model_space =CustomDARTSSpaceK3(input_channels=3, channels=64, num_classes=43, layers=5,verbose =0)
    model_space.set_drop_path_prob(0.2)

    checkpoint_callback = ModelCheckpoint(
        monitor='train_acc',  
        dirpath='./checkpoints', 
        filename='best-checkpoint',  
        save_top_k=1,
        mode='max'  
    )


    evaluator = Lightning(
        AuxLossClassificationModule(1e-3, 3e-4, 0., 600),
        Trainer(
            accelerator="auto",
            callbacks=[checkpoint_callback],  
            max_epochs=600
        ),
        train_dataloaders=train_loader,
        val_dataloaders=valid_loader
    )

    strategy = DartsStrategy(gradient_clip_val=0.3)

    experiment = NasExperiment(model_space, evaluator, strategy)
    experiment.run()
    return experiment



In [ ]:
experiment_results = search("./",16)

In [ ]:
exported_arch = experiment_results.export_top_models(formatter='dict')[0]

exported_arch

In [ ]:
checkpoint_path = './checkpoints/best-checkpoint-v9.ckpt'

checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))

model_checkpoint = CustomDARTSSpaceK3(input_channels=3, channels=64, num_classes=43, layers=5,verbose =0)

model_state_dict = model_checkpoint.state_dict()
pretrained_state_dict = checkpoint['state_dict']

filtered_state_dict = {k: v for k, v in pretrained_state_dict.items() if k in model_state_dict}

model_state_dict.update(filtered_state_dict)

model_checkpoint.load_state_dict(model_state_dict)

torch.save(model_checkpoint,"testmodel.pkt")

if 'global_step' in checkpoint:
    print("Global Step:", checkpoint['global_step'])

if 'callbacks' in checkpoint and isinstance(checkpoint['callbacks'], dict):
    for key, callback in checkpoint['callbacks'].items():
        if isinstance(callback, dict) and 'best_model_score' in callback:
            print("Best Model Score (Train Accuracy):", callback['best_model_score'].item())

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_checkpoint = model_checkpoint.to(device) 

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_checkpoint.parameters(), lr=1e-2, weight_decay=3e-4)

if 'optimizer_state_dict' in checkpoint:
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    print(checkpoint['optimizer_state_dict'])
model_checkpoint.set_drop_path_prob(0.2)
model_checkpoint.train()
def continue_training(model, train_loader, criterion, optimizer, num_epochs, start_epoch=0):
    top_acc = 0.0
    for epoch in range(start_epoch, num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        
        for i, (inputs, labels) in enumerate(train_loader):
            # Check the range of labels before moving to device      
            # Move inputs and labels to the same device as the model
            inputs = inputs.to(device, dtype=torch.float32)  # Ensure inputs are of type float32
            labels = labels.to(device, dtype=torch.long)  # Ensure labels are of type long
            
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(inputs)
            
            # Debug: Check shapes and values

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        epoch_loss = running_loss / len(train_loader)
        accuracy = 100 * correct / total
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.2f}%')
        if accuracy > top_acc:
            top_acc = accuracy
    print('Finished Training')

continue_training(model_checkpoint, train_loader, criterion, optimizer, num_epochs=600, start_epoch=0)
print(f'Top accuracy: {top_acc}')

In [ ]:

def evaluate_model(model, valid_loader, criterion):
    model.eval()  
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(valid_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    valid_loss = running_loss / len(valid_loader)
    accuracy = 100 * correct / total

    print(f'Validation Loss: {valid_loss:.4f}, Validation Accuracy: {accuracy:.2f}%')

    return valid_loss, accuracy

valid_loss, valid_accuracy = evaluate_model(model_checkpoint, valid_loader, criterion)
